In [1]:
# Politico, state by state
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
import os
import numpy as np
import pandas as pd
import pandas as pd
import json
import traceback

import sys
print(sys.version)

3.4.5 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 14:53:07) [MSC v.1600 64 bit (AMD64)]


In [2]:
# Choose what kind of Selenium webdriver to use. This one uses Chrome webdriver; for this to
# work, the Chrome webdriver executable has to be in the system PATH
def make_webdriver(kind='Chrome'):
    if kind == 'Chrome':
        return webdriver.Chrome()
    elif kind == 'nonexistent': # you can put your own here
        #return webdriver.whatevergoeshere
        raise Exception # remove this
    else:
        raise Exception

In [3]:
import logging

log_name = 'politico'
log_dir = '../gitignored'

logger = logging.getLogger(log_name)
log_handler = logging.FileHandler(os.path.join(log_dir, log_name + '.log'))
log_formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
log_handler.setFormatter(log_formatter)
if logger.handlers == []:
    logger.addHandler(log_handler) 
    logger.setLevel(logging.INFO)
logger.info(log_name + 'session started')

In [4]:
use_jupyter_pdb = True
if use_jupyter_pdb:
    from IPython.core.debugger import Tracer
# to activate, use Tracer()()

In [5]:
states = """Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming""".split('\n')

states = [x.strip() for x in states]



In [28]:
%%time
if os.path.isfile('../gitignored/results.json'):
    results = json.load(open('../gitignored/results.json', 'r'))
else:
    results = {}

if os.path.isfile('../gitignored/metadata.json'):
    metadata = json.load(open('../gitignored/metadata.json', 'r'))
else:
    metadata = {}

results = {}
metadata = {}

Wall time: 1e+03 µs


In [29]:
section2urlformat = {'Presidential General Election': 'http://www.politico.com/2016-election/results/map/president/{}/',
                     'Presidential Primaries': 'http://www.politico.com/2016-election/primary/results/map/president/{}/',
                     'Senate General Election': 'http://www.politico.com/2016-election/results/map/senate/{}/',
                     'Senate Primaries': 'http://www.politico.com/2016-election/primary/results/map/senate/{}/',
                     'House General Election': 'http://www.politico.com/2016-election/results/map/house/{}/',
                     'House Primaries': 'http://www.politico.com/2016-election/primary/results/map/house/{}/',
                     'Governor General Election': 'http://www.politico.com/2016-election/results/map/governor/{}/',
                     'Governor Primaries': 'http://www.politico.com/2016-election/primary/results/map/governor/{}/',#
                     #'Ballot Measures': 'http://www.politico.com/2016-election/results/map/ballot-measures/{}/'
                     }

section_names = sorted(list(section2urlformat.keys()))

print(section_names)


['Governor General Election', 'Governor Primaries', 'House General Election', 'House Primaries', 'Presidential General Election', 'Presidential Primaries', 'Senate General Election', 'Senate Primaries']


In [30]:
# use selenium to scroll down to the bottom of the page so that all html is loaded, then process with BeautifulSoup

# to limit the sections or states done in this block (e.g. to fix errors), redefine these variables:
section_names_to_do = None
states_to_do = None
logging_verbose = False
log_errors = False # If False, throws exception

nondefault_items = False

if section_names_to_do is None:
    section_names_to_do = section_names
else:
    nondefault_items = True
if states_to_do is None:
    states_to_do = states
else:
    nondefault_items = True
    
if nondefault_items and logging_verbose:
    logger.info('Verbose logging activated for sections {} and states {}'.format(', '.join(section_names_to_do),
                                                                                     ', '.join(states_to_do)))
results_changed = False
metadata_changed = False

for section in section_names_to_do:
    print('')
    print(section, end=' ')
    logger.info('SECTION {}'.format(section))
    driver = webdriver.Chrome()
    if section not in metadata.keys():
        metadata[section] = {'states_done': [],
                             'states_not_applicable': [],
                             'states_error': {},
                             'states_fips': {}}
        logger.info('metadata blank created')
        metadata_changed = True
    if section not in results.keys():
        logger.info('results blank created')
        results[section] = []  
        results_changed = True
    if logging_verbose:
        logger.info('*VERBOSE: section '+section)
    states_to_ignore = [x for x in states_to_do if x in metadata[section]['states_done'] or x 
                        in metadata[section]['states_not_applicable']]
    for state in states_to_do:
        if state in states_to_ignore:
            if logging_verbose:
                if state in metadata[section]['states_done']:
                    logger.info('*VERBOSE: state {} gitignored because done'.format(state))
                else:
                    logger.info('*VERBOSE: state {} gitignored because not_applicable'.format(state))
            continue
        print(state, end=' ')
        this_results = []
        this_log = "    %line = {{'section': '{}', 'state': '{}', ".format(section,state)
        try:
            state_url = state.lower().replace(' ', '-')
            # make sure this section is in the results
            driver.get('http://www.politico.com/2016-election/results/{}/'.format(state_url))
            time.sleep(10)
            outerhtml = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(outerhtml, 'lxml')
            found = False
            #Tracer()()
            for tag in soup.find_all('a'):
                if tag.text == section:
                    found = True
                if tag.text == section.replace('Primaries', 'Caucuses'):
                    found = True
                if tag.text == section.replace('Primaries', 'Primary'):
                    found = True
                if tag.text == section.replace('Primaries', 'Caucus'):
                    found = True
                if tag.text == section.replace('Primaries', 'Primary and Caucus'):
                    found = True
                if tag.text == section.replace('Primaries', 'Caucus and Primary'):
                    found = True
                if tag.text == section.replace('Primaries', 'Race'):
                    found = True # for Washington's open primary
                if tag.text == section.replace('Primaries', 'Races'):
                    found = True
            if not found:                 
                metadata[section]['states_not_applicable'].append(state)
                logger.info(this_log + "'status': 'not_applicable'}}")
                continue
            # if section found, scrape
            url = section2urlformat[section].format(state_url)
            driver.get(url)
            time.sleep(10) # let it load; very important. There are more robust ways to ensure the
                           # page is loaded, but I was lazy.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            outerhtml = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(outerhtml, 'lxml')
            if section == 'Ballot Measures':
                is_bm = True
                name_type = 'ballot_measure_type'
            else:
                name_type = 'geo_name'
                is_bm = False
            for n_article, article in enumerate(soup.find_all('article', attrs={'class': 'results-group'})):
                fips = article.attrs['data-fips']
                header_name = article.find_all('div', attrs={'class': 'title'})[0].text.strip()
                if state == 'Iowa': # really unprofessional quick fix because I'm tired...
                    header_name.replace('O&apos;Brien', "O'Brien")
                reporting_pct = None
                if section == 'Presidential General Election':
                    if fips == '0':
                        text = article.find_all('p', attrs={'class': 'reporting'})[0].text
                        electoral_votes = int(re.search('Electoral Votes: (\d+)', text).group(1))
                    else:
                        electoral_votes = None
                # figure out if divided into parties, such as primaries/caucuses
                subdivisions = article.find_all('div', attrs={'class': 'results-data'})
                if 'pos-omega' in subdivisions[0].attrs['class']:
                    subdivisions = [article]
                    has_subdivisions = False
                    party = None
                    reporting_pct = float(re.search('(^.+)%', article.find_all('p',
                                                attrs={'class': 'reporting'})[0].text).group(1))
                else:
                    has_subdivisions = True
                for subdiv in subdivisions:
                    skip = False
                    if has_subdivisions:
                        if (section=="House Primaries" and state=='Illinois' and 
                              (header_name.find('5th') != -1 or
                               header_name.find('12th') != -1)):
                            skip = True
                        else:
                            text = subdiv.find_all('div', attrs={'class': 'title'})[0].text.strip()
                            found = re.search('(\D+)([\.\d]+)% Reporting', text)
                            # Politico shows "no county level results will be reported for"
                            # the following primaries/caucuses:
                            if ((not found and state=='Colorado' and text=='Republican') or
                                (not found and state=='Wyoming' and text=='Republican') or
                                (not found and state=='Maine' and text=='Republican')):
                                skip = True
                            else: 
                                party = found.group(1).strip()
                                reporting_pct = float(found.group(2))                    
                    if not skip and not is_bm:
                        table = subdiv.find_all('table', attrs={'class': 'results-table'})[0]
                        has_winner = False
                        winners = subdiv.find_all('span', attrs={'class': 'token token-winner'})
                        if len(winners) > 0:
                            has_winner = True
                        for i, tr in enumerate(table.find_all('tr')):
                            # see note on Politico's individual party a few lines later
                            individual_party = tr.attrs['class'][0]
                            individual_party = individual_party.replace('type-', '')
                            individual_party = individual_party.replace('independent', 'independent_or_other')
                            row = {'state': state, name_type: header_name, 'fips': fips, 'rank': i+1,
                                   'party': party, 'reporting_pct': reporting_pct}
                            row['individual_party'] = individual_party
                            if section == 'Presidential General Election':
                                row['electoral_votes'] = electoral_votes
                            # Politico's one-letter abbreviations for these parties is all screwed up. Replacing
                            # with Democratic/Republican/Other, unfortunately
#                             abbrs = tr.find_all('abbr')
#                             if len(abbrs) > 0:
#                                 individual_party = tr.find_all('abbr')[0].text
#                                 assert len(individual_party) > 0
#                                 row['individual_party'] = individual_party



                            if has_winner:
                                winner = tr.find_all('span', attrs={'class': 'token token-winner'})
                                if len(winner) > 0:
                                    is_winner = True
                                else:
                                    is_winner = False
                                row['is_winner'] = is_winner
                            name = str(tr.find_all('span', attrs={'class': 'name-combo'})[0].contents[-1]).strip()
                            assert len(name) > 0
                            found = re.search('(^.+ )\([iI]\)$', name.strip())
                            if found:
                                row['name'] = found.group(1).strip()
                                row['is_incumbent'] = True
                            else:
                                row['name'] = name.strip()
                                row['is_incumbent'] = False
                            vote_pct = tr.find_all('td', attrs={'class': 'results-percentage'})
                            if len(vote_pct) == 0:
                                row['vote_pct'] = None
                            else:
                                vote_pct = vote_pct[0].text
                                if len(vote_pct) == 0:
                                    row['vote_pct'] = None
                                else:
                                    vote_pct = vote_pct.replace('%', '')
                                    try:
                                        row['vote_pct'] = float(vote_pct)
                                    except ValueError:
                                        row['vote_pct'] = vote_pct # string e,g, 'Uncontested
                            votes = tr.find_all('td', attrs={'class': 'results-popular'})
                            if len(votes) == 0:
                                row['votes'] = None
                            else:
                                votes = votes[0].text
                                if len(votes) == 0:
                                    row['votes'] = None
                                else:
                                    votes = votes.replace(',', '')
                                    try:
                                        row['votes'] = int(votes)
                                    except ValueError:
                                        row['votes'] = votes # string e,g, 'Uncontested'
                            delegates = tr.find_all('td', attrs={'class': 'delegates-cell'})
                            if len(delegates) > 0:
                                delegates = delegates[0].text
                                if len(delegates) > 0:
                                    delegates = int(delegates.replace(',', ''))
                                    row['delegates'] = delegates
                            this_results.append(row)
                    elif not skip: # ballot measure
                        row = {}
                        for detail in subdiv.find_all('div', attrs={'class': 'results-detail'}):
                            title = detail.find_all('h6')[0].text.strip()
                            summary = detail.find_all('p', attrs={'class': 'summary'})[0].text.strip()
                            table = detail.find_all('table', attrs='results-table')[0]
                            for tr in table.find_all('tr', attrs='type-ballot'):
                                row = {'state': state, name_type: header_name, 'fips': fips,
                                      'reporting_pct': reporting_pct, 'title': title, 'summary': summary}
                                text = tr.find_all('span', attrs={'class': 'name-combo'})[0].text
                                if 'Winner' in text:
                                    row['is_winner'] = True
                                    row['choice'] = text.replace('Winner', '').strip()
                                else:
                                    row['is_winner'] = False
                                    row['choice'] = text.strip()
                                row['vote_pct'] = float(tr.find_all('span', attrs={'class': 'number'})[0].text.replace('%', ''))
                                row['votes'] = int(tr.find_all('td', attrs={'class': 'results-popular'})[0].text.replace(',', ''))
                                this_results.append(row)
                                
                    for row in this_results:
                        results[section].append(row)
                        results_changed = True

                    if state not in metadata[section]['states_fips'].keys():
                        metadata[section]['states_fips'][state] = [fips]
                    else:
                        metadata[section]['states_fips'][state].append(fips)
                    metadata_changed = True
            logger.info(this_log+ "'status': 'done', 'fips': {}, 'row': {}}}".format(n_article+1, repr(row)))
            metadata[section]['states_done'].append(state)
            metadata_changed = True
        except:
            print('<-err', end=' ')
            err_type, err_val, sys_tb_object = sys.exc_info()
            error_message = [str(err_type).replace('<class \'', '').replace('\'>', ''), err_val.__doc__]
            for i, tbtext in enumerate(traceback.format_tb(sys_tb_object)):
                toret = tbtext.split('\n')
                error_message.append(str(i))
                error_message.append(toret[0].strip())
                error_message.append(toret[1].strip())
            error_message = ' | '.join(error_message)
            if log_errors:
                logger.info(this_log+"'status': error', 'error_message': {}".format(repr(error_message)))
                metadata[section]['states_error'][state] = error_message
                metadata_changed = True
            else:
                print(error_message)
                raise Exception
    driver.quit()
    
if results_changed:
    with open('../gitignored/results.json', 'w+') as f:
        f.write(json.dumps(results))
if metadata_changed:
    with open('../gitignored/metadata.json', 'w+') as f:
        f.write(json.dumps(metadata))
        
if logging_verbose:
    logging.verbose = False
    logger.info('Verbose logging deactivated')
    
# create dataframes from results, drop duplicate results
dfs = {}
for section in section_names:
    dfs[section] = pd.DataFrame(results[section]).drop_duplicates()
    results[section] = dfs[section].to_dict(orient='records')
with open('results.json', 'w+') as f:
    f.write(json.dumps(results))


Governor General Election Alabama Alaska Arizona Arkansas California Colorado Connecticut Delaware District of Columbia Florida Georgia Hawaii Idaho Illinois Indiana Iowa Kansas Kentucky Louisiana Maine Maryland Massachusetts Michigan Minnesota Mississippi Missouri Montana Nebraska Nevada New Hampshire New Jersey New Mexico New York North Carolina North Dakota Ohio Oklahoma Oregon Pennsylvania Rhode Island South Carolina South Dakota Tennessee Texas Utah Vermont Virginia Washington West Virginia Wisconsin Wyoming 
Governor Primaries Alabama Alaska Arizona Arkansas California Colorado Connecticut Delaware District of Columbia Florida Georgia Hawaii Idaho Illinois Indiana Iowa Kansas Kentucky Louisiana Maine Maryland Massachusetts Michigan Minnesota Mississippi Missouri Montana Nebraska Nevada New Hampshire New Jersey New Mexico New York North Carolina North Dakota Ohio Oklahoma Oregon Pennsylvania Rhode Island South Carolina South Dakota Tennessee Texas Utah Vermont Virginia Washington

In [31]:
# create dataframes from results, drop duplicate results
dfs = {}
for section in section_names:
    dfs[section] = pd.DataFrame(results[section]).drop_duplicates()
    results[section] = dfs[section].to_dict(orient='records')
with open('results.json', 'w+') as f:
    f.write(json.dumps(results))

In [34]:
# fix spelling of O'Brien county
dfs['Presidential Primaries']['geo_name'] = dfs['Presidential Primaries'].\
    geo_name.apply(lambda x: "O'Brien County" if x == 'O&apos;Brien County' else x)

In [35]:
# Check consistencies in geographical names in Presidential Primaries vs Presidential General Election:
df1 = dfs['Presidential Primaries']
df2 = dfs['Presidential General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Kansas':
            print('\nKansas primary results missing from Politico')
        elif state == 'North Dakota':
            print('\nNorth Dakota primary results missing from Politico')
        elif state == 'Minnesota':
            print("\nMinnesota's presidential primaries are by congressional district, vote is by county")
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)



Kansas primary results missing from Politico

Minnesota's presidential primaries are by congressional district, vote is by county

North Dakota primary results missing from Politico

South Dakota
Primary only: Shannon County
General only: Oglala Lakota County


In [36]:
# Check consistencies in geographical names in Senate Primaries vs Senate General Election:

def fix_name(text):
    return text.replace('Primary', '').replace('Primaries', '').replace('Election', '').strip()

df1 = dfs['Senate Primaries']
df2 = dfs['Senate General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Louisiana':
            print('No page for Senate primaries exists on Politico for Louisiana')
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)



Iowa
Primary only: O&apos;Brien County
General only: O'Brien County
No page for Senate primaries exists on Politico for Louisiana

South Dakota
Primary only: Shannon County
General only: Oglala Lakota County


In [13]:
# Check consistencies in geographical names in House Primaries vs General Election:

def fix_name(text):
    return text.replace('Primary', '').replace('Primaries', '').replace('Election', '').strip()

df1 = dfs['House Primaries']
df2 = dfs['House General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([fix_name(x) for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([fix_name(x) for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Louisiana':
            print('No page for House primaries exists on Politico for Louisiana')
        elif state == 'Virginia':
            print('Politico only has House primary results for 2nd, 4th and 6th districts in Virginia')
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)


No page for House primaries exists on Politico for Louisiana
Politico only has House primary results for 2nd, 4th and 6th districts in Virginia


In [37]:
# Check consistencies in geographical names in Governor Primaries vs General Election:
df1 = dfs['Governor Primaries']
df2 = dfs['Governor General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if False:
            pass
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)



Oregon
General only: Baker County, Benton County, Clackamas County, Clatsop County, Columbia County, Coos County, Crook County, Curry County, Deschutes County, Douglas County, Gilliam County, Grant County, Harney County, Hood River County, Jackson County, Jefferson County, Josephine County, Klamath County, Lake County, Lane County, Lincoln County, Linn County, Malheur County, Marion County, Morrow County, Multnomah County, Polk County, Sherman County, Tillamook County, Umatilla County, Union County, Wallowa County, Wasco County, Washington County, Wheeler County, Yamhill County
Last in primaries: None
Last in general: Yamhill County


In [38]:
# cross check presidential and senate general election (for states that had a senate election):
df1 = dfs['Senate General Election']
df2 = dfs['Presidential General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    if len(counties1) > 0:
        df2_ = df2[df2.state == state]
        counties2 = list(df2_.geo_name.unique())
        counties2 = set([x for x in counties2 if x.find('Overall') == -1])
        if counties1 != counties2:
            if False:
                pass
            else:
                print('')
                print(state)
                diff1 = counties1.difference(counties2)
                diff2 = counties2.difference(counties1)
                if len(diff1) > 0:
                    print('Senate only: '+', '.join(sorted(list(diff1))))
                if len(diff2) > 0:
                    print('Presidential only: '+', '.join(sorted(list(diff2))))
                try:
                    last1 = sorted(list(counties1))[-1]
                except:
                    last1 = 'None'
                try:
                    last2 = sorted(list(counties2))[-1]
                except:
                    last2 = 'NONE'
                if last1 != last2:
                    print('Last in Senate: '+last1)
                    print('Last in Presidential: '+last2)


In [16]:
# to redo a section due to a data issue, use this function to remove it from the metadata

def remove_metadata(section, category, state):
    global metadata
    if type(metadata[section][category]) == dict:
        assert state in metadata[section][category].keys()
        metadata[section][category] = {k: v for k, v in metadata[section][category].items() if k != state}
    else:
        assert state in metadata[section][category]
        metadata[section][category].remove(state)
    logger.info('REMOVED: {} | {} | {}'.format(section, category, state))
        

In [39]:
for section in section_names:
    for k, v in metadata[section].items():
        print(section, k, len(v))

Governor General Election states_fips 12
Governor General Election states_not_applicable 39
Governor General Election states_error 0
Governor General Election states_done 12
Governor Primaries states_fips 11
Governor Primaries states_not_applicable 39
Governor Primaries states_error 0
Governor Primaries states_done 12
House General Election states_fips 50
House General Election states_not_applicable 1
House General Election states_error 0
House General Election states_done 50
House Primaries states_fips 49
House Primaries states_not_applicable 2
House Primaries states_error 0
House Primaries states_done 49
Presidential General Election states_fips 51
Presidential General Election states_not_applicable 0
Presidential General Election states_error 0
Presidential General Election states_done 51
Presidential Primaries states_fips 51
Presidential Primaries states_not_applicable 0
Presidential Primaries states_error 0
Presidential Primaries states_done 51
Senate General Election states_fips 

In [42]:
# Other inconsistencies

remove_from_primary_na = False

for state in states:
    errors = []
    
    for section in section_names:
        found_in = []
        if state in metadata[section]['states_error'].keys():
            errors.append('ERROR '+section+' '+metadata[section]['states_error'][state])
            found_in.append('errors')
        if state in metadata[section]['states_done']:
            found_in.append('done')
        if state in metadata[section]['states_not_applicable']:
            found_in.append('not_applicable')
        if len(found_in) == 0:
            errors.append('not found in {} metadata'.format(section))
        elif len(found_in) > 1:
            errors.append('found in {} metadata in mutually exclusive categories: '.format(section,
                                ','.join(found_in)))
        if (state in metadata[section]['states_fips'].keys() and ('done' not in found_in and 'errors'
                not in found_in)):
            errors.append('found in {} states_fips but not in states_done or states_errors'.format(section))
        
#     if (state in metadata['Ballot Measures']['states_done'] and
#         len( metadata['Ballot Measures']['states_fips']) != 2):
#         '{} Ballot Measures tables'.format(len( metadata['Ballot Measures']['states_fips']))
    if (state in metadata['Governor General Election']['states_done'] and not
        state in metadata['Governor Primaries']['states_done']):
        errors.append('Governor election but no primary')
        if remove_from_primary_na:
            remove_metadata('Governor Primaries', 'states_not_applicable', state)
    if (state in metadata['Governor Primaries']['states_done'] and not
        state in metadata['Governor General Election']['states_done']):
        errors.append('Governor primary but no election')
    if (state in metadata['House General Election']['states_done'] and not
        state in metadata['House Primaries']['states_done']):
        errors.append('House election but no primary')
        if remove_from_primary_na:
            remove_metadata('House Primaries', 'states_not_applicable', state)
    if (state in metadata['House Primaries']['states_done'] and not
        state in metadata['House General Election']['states_done']):
        errors.append('House primary but no election')
    if (state in metadata['Senate General Election']['states_done'] and not
        state in metadata['Senate Primaries']['states_done']):
        errors.append('Senate election but no primary')
        if remove_from_primary_na:
            remove_metadata('Senate Primaries', 'states_not_applicable', state)
    if (state in metadata['Senate Primaries']['states_done'] and not
        state in metadata['Senate General Election']['states_done']):
        errors.append('Senate primary but no election')
    if (state in metadata['Presidential General Election']['states_done'] and not
        state in metadata['Presidential Primaries']['states_done']):
        if remove_from_primary_na:
            remove_metadata('Presidential Primaries', 'states_not_applicable', state)
        errors.append('Presidential election but no primary')
    if (state in metadata['Presidential Primaries']['states_done'] and not
        state in metadata['Presidential General Election']['states_done']):
        errors.append('Presidential primary but no election')
    try:
        n1 = len(metadata['Presidential General Election']['states_fips'][state])
        n2 = len(metadata['Senate General Election']['states_fips'][state])
        if n1 != n2:
            errors.append('{} tables in Presidential General Election, {} in Senate'.format(n1, n2))
    except KeyError:
        pass
    try:
        n1 = len(metadata['Presidential Primaries']['states_fips'][state])
        n2 = len(metadata['Senate Primaries']['states_fips'][state])
        if n1 != n2:
            if state not in ['Alaska']: #manual
                errors.append('{} tables in Presidential Primaries, {} in Senate'.format(n1, n2))
    except KeyError:
        pass
    if len(errors) > 0:
        print('')
        print(state)
        print('-'*len(state))
        for error in errors:
            print(error)


California
----------
118 tables in Presidential Primaries, 59 in Senate

Colorado
--------
119 tables in Presidential Primaries, 130 in Senate

Florida
-------
136 tables in Presidential Primaries, 204 in Senate

Hawaii
------
10 tables in Presidential Primaries, 25 in Senate

Kansas
------
2 tables in Presidential Primaries, 212 in Senate

Louisiana
---------
House election but no primary
Senate election but no primary

Missouri
--------
252 tables in Presidential Primaries, 464 in Senate

New York
--------
126 tables in Presidential Primaries, 189 in Senate

North Dakota
------------
1 tables in Presidential Primaries, 162 in Senate

Oklahoma
--------
156 tables in Presidential Primaries, 234 in Senate

Utah
----
60 tables in Presidential Primaries, 120 in Senate

Washington
----------
80 tables in Presidential Primaries, 40 in Senate

Wisconsin
---------
146 tables in Presidential Primaries, 219 in Senate


In [19]:
# all of the above inconsistencies have been investigated, and they are
# not misrepresentations of the data as reported by Politico.

In [43]:
def remove_unvarying_columns(df):
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df = df.drop(col, axis=1)
    return df

In [44]:
# download county demographic data from Deleetdk on Github
r = requests.get('https://raw.githubusercontent.com/Deleetdk/USA.county.data/master/inst/ext/tidy_data.csv')
with open('../gitignored/counties.csv', 'w+', encoding='utf-8') as f:
    f.write(r.text)
dfc = pd.read_csv('../gitignored/counties.csv').dropna(subset=['fips'])
# Remove Kalawao County, fips 15005 pop. 90; they vote in a neighboring county
dfc = dfc[dfc.fips != 15005]
# Remove Bedford city, Virginia county, fips 51515 they have been incorporated into Bedford County
dfc = dfc[dfc.fips != 51515]

# Between the primaries and the election, Shannon County, South Dakota, fips 46113, became Oglala Lakota County, fips 46102
assert sum(dfc.fips == 46113) == 1
dfc_old = dfc.copy()
dfc.loc[dfc.fips == 46113, 'fips'] = 46102
dfc.loc[dfc.fips == 46113, 'County'] = 'Oglala Lakota County, South Dakota'


In [47]:
# save individual csvs
dfs2 = {}
# df = remove_unvarying_columns(dfs['Ballot Measures'])
# df.to_csv('../data/ballot_measures_2016.csv', index=False)
# dfs2['ballot_measures'] = df.copy()
for section in ['Presidential General Election',
 'Presidential Primaries',
 'Senate General Election',
 'Senate Primaries',
 'Governor General Election',
 'Governor Primaries']:
    df = dfs[section]
    df1 = df[df.geo_name.str.contains('Overall')]
    df1 = remove_unvarying_columns(df1)
    assert len(df1) > 0
    df2 = df[~(df.geo_name.str.contains('Overall'))]
    df2 = remove_unvarying_columns(df2)
    name = section.lower().replace(' ', '_') + '_2016'
    df1.to_csv('../data/'+name+'.csv', index=False)
    df2.to_csv('../data/'+name+'_by_county.csv', index=False)
    dfs2[name] = df1.copy()
    dfs2[name+'_by_county.csv'] = df2.copy()
for section in ['House General Election', 'House Primaries']:
    df = remove_unvarying_columns(dfs[section])
    name = section.lower().replace(' ', '_') + '_2016'
    df.to_csv('../data/'+name+'.csv', index=False)
    dfs2[name] = df.copy()
    
# ignore warning, it's fine

In [48]:
dfs2names = sorted(list(dfs2.keys()))

In [49]:
# describe dataframes one by one
n = 0
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))
n+=1

governor_general_election_2016
len=43
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 12
    Overall 2016 Delaware Governor Election Results, Overall 2016 Indiana Governor Election Results, Overall 2016 Missouri Governor Election Results, Overall 2016 Montana Governor Election Results, Overall 2016 New Hampshire Governor Election Results, Overall 2016 North Carolina Governor Election Results, Overall 2016 North Dakota Governor Election Results, Overall 2016 Oregon Governor Election Results, Overall 2016 Utah Governor Election Results, Overall 2016 Vermont Governor Election Results, Overall 2016 Washington Governor Election Results, Overall 2016 West Virginia Governor Election Results
  most common: Overall 2016 Oregon Governor Election Results (5 /  11.6%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    democrat, republican, independent_or_other
  most common: independent_or_other (19 /  44.2%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    Fals

In [50]:
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

governor_general_election_2016_by_county.csv
len=2211
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 602
  most common: 29085 (5 /   0.2%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 469
  most common: Lincoln County (23 /   1.0%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    democrat, republican, independent_or_other
  most common: independent_or_other (1007 /  45.5%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (1951 /  88.2%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 43
  most common: C. Koster (115 /   5.2%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 5
    1, 2, 3, 4, 5
  most common: 1 (602 /  27.2%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 46
  most common: 100.0 (2050 /  92.7%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 12
    Delaware, Indiana, Missouri, Montana, New Hampshire, North Carolina, North Dakota, Oregon, Utah, Vermont, Washington, West Virgini

In [51]:
n += 1

In [52]:
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

governor_primaries_2016
len=65
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 11
    Overall Delaware Governor Primary Results, Overall Indiana Governor Primaries Results, Overall Missouri Governor Races Results, Overall Montana Governor Races Results, Overall New Hampshire Governor Primaries Results, Overall North Carolina Governor Primaries Results, Overall North Dakota Governor Races Results, Overall Utah Governor Races Results, Overall Vermont Governor Primaries Results, Overall Washington Governor Race Results, Overall West Virginia Governor Primaries Results
  most common: Overall Washington Governor Race Results (11 /  16.9%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: republican (28 /  43.1%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (60 /  92.3%)
is_winner
  dtype: object
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: Fals

In [54]:
n += 1

In [55]:
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

governor_primaries_2016_by_county.csv
len=3163
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 566
  most common: 53037 (11 /   0.3%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 447
  most common: Franklin County (34 /   1.1%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: republican (1437 /  45.4%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (2857 /  90.3%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 75
  most common: J. Brunner (115 /   3.6%)
party
  dtype: object
  null: 325 ( 10.3%)
  unique: 6
    None, Democrat, Republican, Democratic, Independent, Open Primary
  most common: Republican (1314 /  41.5%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 11
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
  most common: 1 (1273 /  40.2%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 25
  most common: 100.0 (2778 /  87.8%)
state
  dtype

In [114]:
n += 1

In [58]:
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

governor_primaries_2016_by_county.csv
len=3163
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 566
  most common: 53037 (11 /   0.3%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 447
  most common: Franklin County (34 /   1.1%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: republican (1437 /  45.4%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (2857 /  90.3%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 75
  most common: J. Brunner (115 /   3.6%)
party
  dtype: object
  null: 325 ( 10.3%)
  unique: 6
    None, Democrat, Republican, Democratic, Independent, Open Primary
  most common: Republican (1314 /  41.5%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 11
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
  most common: 1 (1273 /  40.2%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 25
  most common: 100.0 (2778 /  87.8%)
state
  dtype

In [59]:
# merge with demographic info
df['fips'] = df.fips.apply(lambda x: int(x))
these_fips = set(list(df.fips))
assert len(these_fips) < len(df)
these_states = set(list(df.state))
dfc_ = dfc[dfc.State.isin(these_states)]
dfc['fips'] = dfc.fips.apply(lambda x: int(x))
dfc_fips = set(list(dfc_.fips))
assert len(dfc_.fips) == len(dfc_)
assert len(these_fips.difference(dfc_fips)) == 0
assert len(dfc_fips.difference(these_fips)) == 0
dfm = pd.merge(df, dfc_, on='fips', how='left')
newname = dfs2names[n].replace('.csv', '_with_demog.csv')
dfm.to_csv('../merged_with_demog/'+newname)

In [60]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

house_general_election_2016
len=1119
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 53
  most common: 1 (142 /  12.7%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 53
  most common: 1st District Election (142 /  12.7%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: democrat (420 /  37.5%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (731 /  65.3%)
is_winner
  dtype: object
  null: 4 (  0.4%)
  unique: 3
    True, False, nan
  most common: False (680 /  60.8%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 1104
  most common: D. Smith (2 /   0.2%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 12
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
  most common: 1 (435 /  38.9%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 41
  most common: 100.0 (959 /  85.7%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 50
  most common: Californi

In [61]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

house_primaries_2016
len=1560
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 53
  most common: 1 (201 /  12.9%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 87
  most common: 1st District Primaries (170 /  10.9%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 4
    republican, democrat, independent_or_other, unaffiliated
  most common: republican (799 /  51.2%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (1185 /  76.0%)
is_winner
  dtype: object
  null: 16 (  1.0%)
  unique: 3
    True, False, nan
  most common: True (797 /  51.1%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 1521
  most common: J. Lewis (4 /   0.3%)
party
  dtype: object
  null: 362 ( 23.2%)
  unique: 4
    None, Democratic, Republican, Independent
  most common: Republican (632 /  40.5%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 17
  most common: 1 (750 /  48.1%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 33
  m

In [62]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

presidential_general_election_2016
len=302
delegates
  dtype: float64
  null: 252 ( 83.4%)
  unique: 21
  most common: 3.0 (9 /   3.0%)
electoral_votes
  dtype: float64
  null: 0 (  0.0%)
  unique: 19
  most common: 6.0 (45 /  14.9%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 51
  most common: Overall 2016 Colorado Presidential Election Results (22 /   7.3%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: independent_or_other (200 /  66.2%)
is_winner
  dtype: object
  null: 11 (  3.6%)
  unique: 3
    True, False, nan
  most common: False (242 /  80.1%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 32
  most common: G. Johnson (51 /  16.9%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 22
  most common: 1 (51 /  16.9%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 9
    100.0, 99.8, 96.1, 99.1, 99.9, 99.3, 99.7, 82.5, 81.6
  most common: 100.0 (217 /  71.9%)
state
  dtype: obje

In [63]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

presidential_general_election_2016_by_county.csv
len=18017
delegates
  dtype: float64
  null: 18015 (100.0%)
  unique: 2
    nan, 3.0
  most common: 3.0 (2 /   0.0%)
electoral_votes
  dtype: float64
  null: 18007 ( 99.9%)
  unique: 2
    nan, 3.0
  most common: 3.0 (10 /   0.1%)
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 3112
  most common: 8067 (22 /   0.1%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 1852
  most common: Washington County (179 /   1.0%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: independent_or_other (11791 /  65.4%)
is_winner
  dtype: object
  null: 18007 ( 99.9%)
  unique: 3
    nan, True, False
  most common: False (8 /   0.0%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 32
  most common: D. Trump (3113 /  17.3%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 22
  most common: 3 (3113 /  17.3%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 57
  m

In [64]:
# investigate the ones with delegates
df[pd.notnull(df.delegates)]

,delegates,electoral_votes,fips,geo_name,individual_party,is_winner,name,rank,reporting_pct,state,vote_pct,votes
278,3.0,3.0,0,Alaska,republican,True,D. Trump,1,99.8,Alaska,52.9,130415
2737,3.0,3.0,0,District of Columbia,democrat,True,H. Clinton,1,100.0,District of Columbia,92.8,260223


In [65]:
df[df.geo_name.str.contains('Alaska')]

,delegates,electoral_votes,fips,geo_name,individual_party,is_winner,name,rank,reporting_pct,state,vote_pct,votes
278,3.0,3.0,0,Alaska,republican,True,D. Trump,1,99.8,Alaska,52.9,130415
279,NaN,3.0,0,Alaska,democrat,False,H. Clinton,2,99.8,Alaska,37.7,93007
280,NaN,3.0,0,Alaska,independent_or_other,False,G. Johnson,3,99.8,Alaska,5.9,14593
281,NaN,3.0,0,Alaska,independent_or_other,False,J. Stein,4,99.8,Alaska,1.8,4445
282,NaN,3.0,0,Alaska,independent_or_other,False,D. Castle,5,99.8,Alaska,1.3,3156
283,NaN,3.0,0,Alaska,independent_or_other,False,R. De La Fuente,6,99.8,Alaska,0.4,972


In [66]:
# investigating the Politico website, this appears to be a mistake in the coding.
df = df.drop('delegates', axis=1)

In [67]:
# fix the fips in DC
df.loc[df.state == 'District of Columbia', 'fips'] = 11001

In [74]:
df = df[df.fips != '0'] # Drop the Alaska, not-by-county
df.to_csv('../data/'+dfs2names[n], index=False)

In [75]:
# merge with demographic info
df['fips'] = df.fips.apply(lambda x: int(x))
these_fips = set(list(df.fips))
assert len(these_fips) < len(df)
these_states = set(list(df.state))
dfc_ = dfc[dfc.State.isin(these_states)]
dfc['fips'] = dfc.fips.apply(lambda x: int(x))
dfc_fips = set(list(dfc_.fips))
assert len(dfc_.fips) == len(dfc_)
assert len(these_fips.difference(dfc_fips)) == 0
assert len(dfc_fips.difference(these_fips)) == 0
dfm = pd.merge(df, dfc_, on='fips', how='left')
newname = dfs2names[n].replace('.csv', '_with_demog.csv')
dfm.to_csv('../merged_with_demog/'+newname)

In [76]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

presidential_primaries_2016
len=734
delegates
  dtype: float64
  null: 531 ( 72.3%)
  unique: 69
  most common: 1.0 (17 /   2.3%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 51
  most common: Overall New Hampshire Presidential Primaries Results (60 /   8.2%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 2
    democrat, republican
  most common: republican (479 /  65.3%)
is_winner
  dtype: object
  null: 18 (  2.5%)
  unique: 3
    True, False, nan
  most common: False (619 /  84.3%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 72
  most common: B. Sanders (51 /   6.9%)
party
  dtype: object
  null: 11 (  1.5%)
  unique: 3
    Democratic, Republican, None
  most common: Republican (479 /  65.3%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 31
  most common: 1 (100 /  13.6%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 15
  most common: 100.0 (590 /  80.4%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 51
  most common: New Hampshire

In [77]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

presidential_primaries_2016_by_county.csv
len=44739
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 2875
  most common: 33019 (60 /   0.1%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 1739
  most common: Washington County (400 /   0.9%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    democrat, republican, independent_or_other
  most common: republican (30339 /  67.8%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 77
  most common: H. Clinton (2876 /   6.4%)
party
  dtype: object
  null: 90 (  0.2%)
  unique: 5
    Democratic, Republican, Democrat, None, Independent
  most common: Republican (30339 /  67.8%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 31
  most common: 1 (5669 /  12.7%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 81
  most common: 100.0 (44127 /  98.6%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 49
  most common: Texas (5588 /  12.5%)
vote_pct
  dtype: float64
  null: 10 (  0.0%)
  unique: 912
  most commo

In [78]:
# merge with demographic info
df['fips'] = df.fips.apply(lambda x: int(x))
df = df[df.fips > 9] # Alaska was listed with Congressional Districts, not counties
these_fips = set(list(df.fips))
assert len(these_fips) < len(df)
these_states = set(list(df.state))
dfc_ = dfc_old[dfc_old.State.isin(these_states)] # use the old version because of the South Dakota county that changed
dfc['fips'] = dfc.fips.apply(lambda x: int(x))
dfc_fips = set(list(dfc_.fips))
assert len(dfc_.fips) == len(dfc_)
assert len(these_fips.difference(dfc_fips)) == 0
assert len(dfc_fips.difference(these_fips)) == 0
dfm = pd.merge(df, dfc_, on='fips', how='left')
newname = dfs2names[n].replace('.csv', '_with_demog.csv')
dfm.to_csv('../merged_with_demog/'+newname)

In [79]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

senate_general_election_2016
len=155
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 34
  most common: Overall 2016 Louisiana Senate Election Results (24 /  15.5%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: independent_or_other (73 /  47.1%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (126 /  81.3%)
is_winner
  dtype: object
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (120 /  77.4%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 154
  most common: J. Williams (2 /   1.3%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 24
  most common: 1 (34 /  21.9%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 9
    100.0, 99.8, 96.1, 99.1, 99.9, 99.7, 99.2, 81.8, 82.2
  most common: 100.0 (103 /  66.5%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 34
  most common: Louisiana (24 /  15.5%)
vote_pct
  dtype: float6

In [80]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

senate_general_election_2016_by_county.csv
len=8978
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 2059
  most common: 22091 (24 /   0.3%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 1288
  most common: Washington County (88 /   1.0%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 3
    republican, democrat, independent_or_other
  most common: independent_or_other (3964 /  44.2%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (7174 /  79.9%)
is_winner
  dtype: object
  null: 8972 ( 99.9%)
  unique: 3
    nan, True, False
  most common: False (5 /   0.1%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 154
  most common: J. Barksdale (159 /   1.8%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 24
  most common: 1 (2059 /  22.9%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 55
  most common: 100.0 (8763 /  97.6%)
state
  dtype: object
  null: 0 (  0.0%)
  unique: 34
  most common: Louisiana (1

In [81]:
df = df[df.fips != '0'] # Drop the Alaska, not-by-county
df.to_csv('../data/'+dfs2names[n], index=False)

In [82]:
# merge with demographic info
df['fips'] = df.fips.apply(lambda x: int(x))
df = df[df.fips > 0] # Alaska was listed with Congressional Districts, not counties
these_fips = set(list(df.fips))
assert len(these_fips) < len(df)
these_states = set(list(df.state))
dfc_ = dfc[dfc.State.isin(these_states)] # use the old version because of the South Dakota county that changed
dfc['fips'] = dfc.fips.apply(lambda x: int(x))
dfc_fips = set(list(dfc_.fips))
assert len(dfc_.fips) == len(dfc_)
assert len(these_fips.difference(dfc_fips)) == 0
assert len(dfc_fips.difference(these_fips)) == 0
dfm = pd.merge(df, dfc_, on='fips', how='left')
newname = dfs2names[n].replace('.csv', '_with_demog.csv')
dfm.to_csv('../merged_with_demog/'+newname)

C:\Users\David\Anaconda3\envs\py34\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [83]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

senate_primaries_2016
len=246
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 33
  most common: Overall California Senate Primary Results (34 /  13.8%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 4
    democrat, republican, independent_or_other, unaffiliated
  most common: republican (110 /  44.7%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (217 /  88.2%)
is_winner
  dtype: object
  null: 0 (  0.0%)
  unique: 2
    True, False
  most common: False (167 /  67.9%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 245
  most common: None of these candidates (2 /   0.8%)
party
  dtype: object
  null: 51 ( 20.7%)
  unique: 4
    Democratic, Republican, Independent, None
  most common: Republican (95 /  38.6%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 34
  most common: 1 (77 /  31.3%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 11
    100.0, 98.4, 99.9, 0.0, 99.2, 99.7, 99.1, 93.0, 90.9, 97.6, 

In [84]:
n += 1
print (dfs2names[n])
df = dfs2[dfs2names[n]]
length = len(df)
print ('len={}'.format(len(df)))
for col in df.columns:
    s = df[col]
    print(col)
    print('  dtype: {}'.format(s.dtype))
    print('  null: {} ({:5.1f}%)'.format(sum(pd.isnull(s)), 100.0*sum(pd.isnull(s))/length))
    print('  unique: {}'.format(len(s.unique())))
    if len(s.unique()) < 15:
        print('    {}'.format(', '.join([str(x) for x in s.unique()])))
    vcs = s.value_counts()
    vcs_name = vcs.index[0]
    vcs_count = vcs.iloc[0]
    print('  most common: {} ({} / {:5.1f}%)'.format(vcs_name, vcs_count, 100.0*vcs_count/length))

senate_primaries_2016_by_county.csv
len=13874
fips
  dtype: object
  null: 0 (  0.0%)
  unique: 1995
  most common: 6035 (34 /   0.2%)
geo_name
  dtype: object
  null: 0 (  0.0%)
  unique: 1223
  most common: Washington County (141 /   1.0%)
individual_party
  dtype: object
  null: 0 (  0.0%)
  unique: 4
    democrat, republican, independent_or_other, unaffiliated
  most common: democrat (5872 /  42.3%)
is_incumbent
  dtype: bool
  null: 0 (  0.0%)
  unique: 2
    False, True
  most common: False (12080 /  87.1%)
name
  dtype: object
  null: 0 (  0.0%)
  unique: 296
  most common: J. Coyne (159 /   1.1%)
party
  dtype: object
  null: 2125 ( 15.3%)
  unique: 6
    Democratic, Republican, Democrat, Independent, Open Primary, None
  most common: Republican (4961 /  35.8%)
rank
  dtype: int64
  null: 0 (  0.0%)
  unique: 34
  most common: 1 (4525 /  32.6%)
reporting_pct
  dtype: float64
  null: 0 (  0.0%)
  unique: 40
  most common: 100.0 (12945 /  93.3%)
state
  dtype: object
  null: 0 ( 

In [85]:
df = df[df.fips != '0'] # Drop the Alaska, not-by-county
df.to_csv('../data/'+dfs2names[n], index=False)

In [86]:
# merge with demographic info
df['fips'] = df.fips.apply(lambda x: int(x))
#df = df[df.fips > 9] # Alaska was listed with Congressional Districts, not counties
these_fips = set(list(df.fips))
assert len(these_fips) < len(df)
these_states = set(list(df.state))
dfc_ = dfc_old[dfc_old.State.isin(these_states)] # use the old version because of the South Dakota county that changed
dfc['fips'] = dfc.fips.apply(lambda x: int(x))
dfc_fips = set(list(dfc_.fips))
assert len(dfc_.fips) == len(dfc_)
assert len(these_fips.difference(dfc_fips)) == 0
assert len(dfc_fips.difference(these_fips)) == 0
dfm = pd.merge(df, dfc_, on='fips', how='left')
newname = dfs2names[n].replace('.csv', '_with_demog.csv')
dfm.to_csv('../merged_with_demog/'+newname)

C:\Users\David\Anaconda3\envs\py34\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
